In [1]:
import os
import sys

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch3d
from pytorch3d import _C
from pytorch3d.io import load_obj, save_obj, load_objs_as_meshes
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.ops import sample_points_from_meshes, knn_points, estimate_pointcloud_normals, knn_gather, cubify
from pytorch3d.loss.point_mesh_distance import point_mesh_edge_distance, point_mesh_face_distance
import trimesh

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import pyvista as pv
pv.start_xvfb()
pv.set_jupyter_backend('html')


from torch.utils.data import DataLoader, Dataset, DistributedSampler  
from torch.nn.parallel import DistributedDataParallel as DDP  

# # from pytorch3d.structures import Meshes

# from .utils import one_hot_sparse
from ops.mesh_geometry import *
# import pymeshlab as ml

import mcubes

In [ ]:

trimesh_tem = trimesh.load('./data_example/Bull.obj', force='mesh')

# delete a few faces to make the mesh non-watertight
# trimesh_tem.faces[590:593,:] = trimesh_tem.faces[590:591,:]
print(trimesh_tem.is_watertight)

device = torch.device("cuda:0")

mesh_tem = Meshes(verts=[torch.from_numpy(trimesh_tem.vertices).float()], 
                  faces=[torch.from_numpy(trimesh_tem.faces).long()]).to(device)

mesh_tem = normalize_mesh(mesh_tem) # normalize the mesh to fit in the unit cube

print(mesh_tem.verts_packed().shape)


In [ ]:
pl = pv.Plotter(notebook=True)

trimesh_tem = trimesh.Trimesh(vertices=mesh_tem.verts_list()[0].cpu().numpy(), faces=mesh_tem.faces_list()[0].cpu().numpy())

pl.add_mesh(trimesh_tem, color='lightblue', show_edges=True, opacity=1)


# pl.add_points(coordinates_downsampled.cpu().numpy()[np.random.choice(n_points, 1000)], color='red', point_size=5)

# pl.camera.roll = 10
pl.camera.elevation = 140
pl.camera.azimuth = 60
pl.camera.zoom = 1.3

pl.show() #screenshot='out_exp/tem_mesh_2.png', window_size=[800,800])

Fipped duplication

In [ ]:
## non-closed mesh check

centroids = mesh_tem.verts_packed()[mesh_tem.faces_packed()].mean(-2)

with torch.no_grad():
    #sdf_result = signed_distance_field(mesh_tem, coordinates_downsampled, allow_grad=False)
    occp_result = occupancy(mesh_tem, centroids, allow_grad=False)

non_closed_index = torch.where((occp_result- torch.round(occp_result)).abs() > 1e-5)[0]

## duplicate & flip the non-closed faces 

mesh_verts_new = mesh_tem.verts_packed().clone() - mesh_tem.verts_normals_packed()*1e-2

faces_new = mesh_tem.faces_packed().clone()[non_closed_index,:]

faces_new = faces_new[:,[0,2,1]]

print(faces_new.shape)

# new_mesh = Meshes(verts=[mesh_verts_new], faces=[face_new])

# merge the old and new mesh
new_mesh = Meshes(verts=[mesh_verts_new], faces=[faces_new])

In [ ]:
# voxelizer = Differentiable_Voxelizer(bbox_density=128)


sample_size = 256

meshbbox = mesh_tem.get_bounding_boxes()[0]

coordinates_downsampled = torch.stack(torch.meshgrid(torch.linspace(meshbbox[0,0], meshbbox[0,1], sample_size),
                                                        torch.linspace(meshbbox[1,0], meshbbox[1,1], sample_size),
                                                        torch.linspace(meshbbox[2,0], meshbbox[2,1], sample_size)), dim=-1)

coordinates_downsampled = coordinates_downsampled.view(-1, 3).to(device)

n_points = coordinates_downsampled.shape[0]


In [6]:
with torch.no_grad():
    #sdf_result = signed_distance_field(new_mesh, coordinates_downsampled, allow_grad=False)
    occp_result_ori = occupancy(mesh_tem, coordinates_downsampled, allow_grad=False, max_query_point_batch_size=2000)

    occp_result_inverse = occupancy(new_mesh, coordinates_downsampled, allow_grad=False, max_query_point_batch_size=2000)

In [7]:
occp_result = torch.where((occp_result_ori - occp_result_ori.round()).abs() > 1e-5, occp_result_inverse+occp_result_ori, occp_result_ori)
occpfield = occp_result.view(1, sample_size, sample_size, sample_size)
occpfield = occpfield.permute(0, 3, 2, 1)

Re-cubify

In [8]:

## pymcubes marching cubes
vertices, triangles = mcubes.marching_cubes((occpfield.permute(0, 3, 2, 1))[0].cpu().numpy(), 0.5)


## rescale the vertices to the original mesh
vertices = meshbbox.mean(dim=-1).cpu().numpy() + (vertices*2/(sample_size-1)-1)*((meshbbox[:,1]-meshbbox[:,0]).view(1, 3).cpu().numpy())/2

# create a mesh object
trimesh_cubified = trimesh.Trimesh(vertices=vertices, faces=triangles)


## Pytorch3d cubify
cubified = cubify(occpfield, 0.5) # cubify the voxel grid, which is the inverse operation of voxelization

cubified = cubified.update_padded(meshbbox.mean(dim=-1) + cubified.verts_padded()*((meshbbox[:,1]-meshbbox[:,0]).view(1,1, 3).to(device))/2)

trimesh_cubified = trimesh.Trimesh(cubified.verts_packed().detach().cpu().numpy(), cubified.faces_packed().detach().cpu().numpy())


## smooth the mesh
# trimesh_cubified = trimesh.smoothing.filter_laplacian(trimesh_cubified, iterations=3)
pl = pv.Plotter(notebook=True)

pl.add_mesh(trimesh_tem, color='lightblue', show_edges=True, opacity=0.2)

pl.add_mesh(trimesh_cubified, color='lightgreen', opacity=1, show_edges=False)

pl.camera.elevation = 140
pl.camera.azimuth = 60
pl.camera.zoom = 1.3

pl.show() 
